# NBinomial n=40, p=0.625

In [1]:
import SimFunctions
import SimRNG
import SimClasses
import numpy as np
import pandas as pd
import math
import scipy.stats
from random import randint

def Service_Time(Item_Num):
    # service time ~ normal distribution
    ServiceTime = abs(np.random.normal(1/10*Item_Num + 3))
    return ServiceTime

# number of items ~ negative binomial distribution
def ItemNum():
    return np.random.negative_binomial(40, 0.625)

def Arrival():
    global e
    SimFunctions.Schedule(Calendar,"Arrival",SimRNG.Expon(MeanTBA, 1))
    Customer = SimClasses.Entity()
    NumQueue_list = []
    NumExpress_list = []
    # e=number of Express lanes
    # Calculate the mean of e express checkout queues
    for i in range(e):
        NumExpress_list.append(TheQueues[i].NumQueue())
    NumExpress_mean = np.mean(NumExpress_list)
    
    #calculate the mean of not-express checkout queues
    for i in range(6-e):
        NumQueue_list.append(TheQueues[i+e].NumQueue())
    NumQueue_mean = np.mean(NumQueue_list)
    
    #if the customer has less than 18 Items, he will consider go to self checkout or express counter
    if ItemNum() < 18:
        if Queue_Self.NumQueue() < NumExpress_mean:
            #if the self checkout queue length is less than express queue length
            #the customer will join the self check out queue
            Queue_Self.Add(Customer)  
            if Server_Self.Busy < Server_Self.NumberOfUnits:
                Server_Self.Seize(1)
                SimFunctions.Schedule(Calendar, "EndOfService_Self", Service_Time(ItemNum()))                  
        else: 
            # randomly assign the customer to an Express lane
            # randomly generate an integer from 1 to e
            express = randint(1,e)
            TheQueues[express-1].Add(Customer)
            if TheResources[express-1].Busy == 0:
                TheResources[express-1].Seize(1)
                SimFunctions.Schedule(Calendar,"EndOfService"+str(express),Service_Time(ItemNum()))        
    else:
        if Queue_Self.NumQueue() < NumQueue_mean:
            Queue_Self.Add(Customer)  
            if Server_Self.Busy < Server_Self.NumberOfUnits:
                Server_Self.Seize(1)
                SimFunctions.Schedule(Calendar, "EndOfService_Self", Service_Time(ItemNum()))                  
        else: 
            min_index = NumQueue_list.index(min(NumQueue_list))
            TheQueues[min_index+e].Add(Customer)
            if TheResources[min_index+e].Busy == 0:
                TheResources[min_index+e].Seize(1)
                SimFunctions.Schedule(Calendar,"EndOfService"+str(min_index+e+1),Service_Time(ItemNum()))
                
                
def EndOfService1():
    global Total_Customer1
    Total_Customer1 += 1
    DepartingCustomer1 = Queue1.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer1.CreateTime)
    if Queue1.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService1",Service_Time(ItemNum()))
    else:
        Server1.Free(1)
        
def EndOfService2():
    global Total_Customer2
    Total_Customer2 += 1
    DepartingCustomer2 = Queue2.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer2.CreateTime)
    if Queue2.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService2",Service_Time(ItemNum()))
    else:
        Server2.Free(1)
    
def EndOfService3():
    global Total_Customer3
    Total_Customer3 += 1
    DepartingCustomer3 = Queue3.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer3.CreateTime)
    if Queue3.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService3",Service_Time(ItemNum()))
    else:
        Server3.Free(1)
    
def EndOfService4():
    global Total_Customer4
    Total_Customer4 += 1
    DepartingCustomer4 = Queue4.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer4.CreateTime)
    if Queue4.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService4",Service_Time(ItemNum()))
    else:
        Server4.Free(1)
    
def EndOfService5():
    global Total_Customer5
    Total_Customer5 += 1
    DepartingCustomer5 = Queue5.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer5.CreateTime)
    if Queue5.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService5",Service_Time(ItemNum()))
    else:
        Server5.Free(1)
    
def EndOfService6():
    global Total_Customer6
    Total_Customer6 += 1
    DepartingCustomer6 = Queue6.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer6.CreateTime)
    if Queue6.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService6",Service_Time(ItemNum()))
    else:
        Server6.Free(1)
          
def EndOfService_Self():
    global Total_Customer_Self
    Total_Customer_Self += 1
    DepartingCustomer_Self = Queue_Self.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer_Self.CreateTime)
    if Queue_Self.NumQueue() >= Server_Self.NumberOfUnits:
        SimFunctions.Schedule(Calendar,"EndOfService_Self",Service_Time(ItemNum()))
    else:
        Server_Self.Free(1)

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m-h, m+h        
        
ZSimRNG = SimRNG.InitializeRNSeed()
Queue1 = SimClasses.FIFOQueue()
Queue2 = SimClasses.FIFOQueue()
Queue3 = SimClasses.FIFOQueue()
Queue4 = SimClasses.FIFOQueue()
Queue5 = SimClasses.FIFOQueue()
Queue6 = SimClasses.FIFOQueue()
Queue_Self = SimClasses.FIFOQueue()

Wait = SimClasses.DTStat()
Server1 = SimClasses.Resource()
Server2 = SimClasses.Resource()
Server3 = SimClasses.Resource()
Server4 = SimClasses.Resource()
Server5 = SimClasses.Resource()
Server6 = SimClasses.Resource()
Server_Self = SimClasses.Resource()

Calendar = SimClasses.EventCalendar()

TheCTStats = []
TheDTStats = []
TheQueues = []
AllStats = []
TheResources = []

TheDTStats.append(Wait)
TheQueues.append(Queue1)
TheQueues.append(Queue2)
TheQueues.append(Queue3)
TheQueues.append(Queue4)
TheQueues.append(Queue5)
TheQueues.append(Queue6)
TheQueues.append(Queue_Self)
TheResources.append(Server1)
TheResources.append(Server2)
TheResources.append(Server3)
TheResources.append(Server4)
TheResources.append(Server5)
TheResources.append(Server6)
TheResources.append(Server_Self)

# set number of Express lanes
e = 1
Server1.SetUnits(1)
Server2.SetUnits(1)
Server3.SetUnits(1)
Server4.SetUnits(1)
Server5.SetUnits(1)
Server6.SetUnits(1)
Server_Self.SetUnits(6)

MeanTBA = 1.0 #a customer will arrive 1 min
T = 840 # 8AM-10M=14hrs

for reps in range(0,35,1):
    Total_Customer1 = 0
    Total_Customer2 = 0
    Total_Customer3 = 0
    Total_Customer4 = 0
    Total_Customer5 = 0
    Total_Customer6 = 0
    Total_Customer_Self = 0
    
    SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
    SimFunctions.Schedule(Calendar,"Arrival",SimRNG.Expon(MeanTBA, 1))
    SimFunctions.Schedule(Calendar,"EndSimulation",T)
    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    
    if NextEvent.EventType == "Arrival":
        Arrival()
    elif NextEvent.EventType == "EndOfService1":
        EndOfService1()
    elif NextEvent.EventType == "EndOfService2":
        EndOfService2()
    elif NextEvent.EventType == "EndOfService3":
        EndOfService3()
    elif NextEvent.EventType == "EndOfService4":
        EndOfService4()
    elif NextEvent.EventType == "EndOfService5":
        EndOfService5()
    elif NextEvent.EventType == "EndOfService6":
        EndOfService6()
    elif NextEvent.EventType == "EndOfService_Self":
        EndOfService_Self()

    while NextEvent.EventType != "EndSimulation":
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Arrival":
            Arrival()
        elif NextEvent.EventType == "EndOfService1":
            EndOfService1()
        elif NextEvent.EventType == "EndOfService2":
            EndOfService2()
        elif NextEvent.EventType == "EndOfService3":
            EndOfService3()
        elif NextEvent.EventType == "EndOfService4":
            EndOfService4()
        elif NextEvent.EventType == "EndOfService5":
            EndOfService5()
        elif NextEvent.EventType == "EndOfService6":
            EndOfService6()
        elif NextEvent.EventType == "EndOfService_Self":
            EndOfService_Self()
    
    AllStats.append([Wait.Mean(),Total_Customer1+Total_Customer2+Total_Customer3+Total_Customer4+Total_Customer5+Total_Customer6+Total_Customer_Self])


Results = pd.DataFrame(AllStats, columns=["Wait","Total Customer"])
print(Results)
print("\nFor n=35:")
print("Average Waiting Time:", Results.loc[:,"Wait"].mean())
print("With 95% CI:", mean_confidence_interval(Results.loc[:, "Wait"]))
print("Average Total Customer:",Results.loc[:,"Total Customer"].mean())

        Wait  Total Customer
0   6.214346             811
1   6.170069             799
2   6.486981             860
3   6.444298             828
4   6.377064             838
5   6.442597             842
6   6.436134             828
7   6.185301             813
8   6.280687             851
9   6.300738             849
10  6.704931             860
11  6.061890             826
12  6.049844             804
13  6.627489             886
14  6.509885             906
15  6.317150             830
16  6.103915             788
17  6.573390             891
18  6.087208             805
19  6.220667             830
20  6.340680             839
21  6.456690             817
22  6.170655             811
23  6.037068             804
24  6.214917             772
25  6.546075             897
26  6.373325             803
27  6.366643             862
28  6.240609             819
29  6.430163             891
30  6.467195             820
31  6.230775             801
32  6.136127             834
33  6.432719  

In conclusion, even if there is an express lane, the average waiting time did not decrease, the total performance did not improve. The reason may be that customers are likely to buy more items on weekends. The express lane did not make a difference but occupied a standard counter, which turns out to be useless. We suggest not using express on weekends or adding express counters with keeping all the standard checkout counters.